<h1 style="color:#e62f2f"><i> LA Crime Analysis </i></h1>
<br>
<h3>Dataset source link</h3>
<a href="https://catalog.data.gov/dataset/crime-data-from-2020-to-present">https://catalog.data.gov/dataset/crime-data-from-2020-to-present</a> <br>
<a href="https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8">https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8</a>

### **1. Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mplt
import string, re
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from geopy.geocoders import Nominatim
from geopy.distance import distance
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go

### **2. Import Dataset**

In [2]:
rawData = pd.read_csv('./crime data.csv')
rawData.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


### **3. Duplicating data and performing Data Cleansing**

In [3]:
rawDataCP = rawData
rawDataCP

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654351,230804108,01/03/2023 12:00:00 AM,01/02/2023 12:00:00 AM,1130,8,West LA,834,2,956,"LETTERS, LEWD - TELEPHONE CALLS, LEWD",...,IC,Invest Cont,956.0,NaN,NaN,NaN,10500 WILSHIRE BL,NaN,34.0643,-118.4321
654352,230104026,01/01/2023 12:00:00 AM,01/01/2023 12:00:00 AM,30,1,Central,119,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,800 N ALAMEDA ST,NaN,34.0561,-118.2375
654353,231104474,01/12/2023 12:00:00 AM,01/12/2023 12:00:00 AM,1240,11,Northeast,1107,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,IC,Invest Cont,440.0,NaN,NaN,NaN,1000 MILWAUKEE AV,NaN,34.1214,-118.1915
654354,230804266,01/08/2023 12:00:00 AM,01/08/2023 12:00:00 AM,1030,8,West LA,839,1,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",...,IC,Invest Cont,341.0,NaN,NaN,NaN,10200 SANTA MONICA BL,NaN,34.0611,-118.4184


**3.1 Deleting not useful columns**

In [4]:
# List of columns
rawDataCP.columns = rawDataCP.columns.str.lower()
rawDataCP.columns

Index(['dr_no', 'date rptd', 'date occ', 'time occ', 'area', 'area name',
       'rpt dist no', 'part 1-2', 'crm cd', 'crm cd desc', 'mocodes',
       'vict age', 'vict sex', 'vict descent', 'premis cd', 'premis desc',
       'weapon used cd', 'weapon desc', 'status', 'status desc', 'crm cd 1',
       'crm cd 2', 'crm cd 3', 'crm cd 4', 'location', 'cross street', 'lat',
       'lon'],
      dtype='object')

In [5]:
#rawDataCP.drop(['dr_no', 'date rptd', 'rpt dist no','part 1-2','mocodes', 'vict descent','weapon used cd', 'weapon desc', 'status', 'status desc', 'crm cd 1', 'crm cd 2', 'crm cd 3', 'crm cd 4'],axis=1,inplace=True)
rawDataCP.drop(['dr_no', 'date rptd', 'rpt dist no','part 1-2','mocodes', 'vict descent', 'status', 'status desc', 'crm cd 1', 'crm cd 2', 'crm cd 3', 'crm cd 4'],axis=1,inplace=True)
rawDataCP.columns

Index(['date occ', 'time occ', 'area', 'area name', 'crm cd', 'crm cd desc',
       'vict age', 'vict sex', 'premis cd', 'premis desc', 'weapon used cd',
       'weapon desc', 'location', 'cross street', 'lat', 'lon'],
      dtype='object')

**3.2 Removing extra spaces from values of 'LOCATION' column and merging the 'Cross Street' column to it.**

In [6]:
rawDataCP['location'].values

array(['1100 W  39TH                         PL',
       '700 S  HILL                         ST',
       '200 E  6TH                          ST', ...,
       '1000    MILWAUKEE                    AV',
       '10200    SANTA MONICA                 BL',
       '12500    BRANFORD                     ST'], dtype=object)

In [7]:
rawDataCP["location"] = (rawDataCP["location"] + ' ' + rawDataCP["cross street"].fillna('')).str.strip().replace(r'\s+', ' ', regex=True)
rawDataCP["location"].head(100).values

array(['1100 W 39TH PL', '700 S HILL ST', '200 E 6TH ST',
       '5400 CORTEEN PL', '14400 TITUS ST', '700 S BROADWAY',
       '700 S FIGUEROA ST', '200 E 6TH ST', '700 BERNARD ST',
       '11900 BALBOA BL', '15TH OLIVE', '800 N ALAMEDA ST',
       '800 S OLIVE ST', '700 W 7TH ST', '100 S LOS ANGELES ST',
       '14200 BERG ST', '3200 W AVENUE 32', 'PACIFIC COAST VERMONT',
       '14700 FRIAR ST', '7TH HILL', '13600 LEADWELL ST', '700 W 7TH ST',
       '700 W 7TH ST', '5700 ENFIELD AV', '600 SAN JULIAN ST',
       '18600 COLLINS ST', '2700 N VERMONT AV', 'ALAMEDA LOS ANGELES',
       '100 S ACARI DR', '300 S FIGUEROA ST', '700 N MAIN ST',
       '8500 CASHIO ST', '3200 W AVENUE 32', '500 N FIGUEROA ST',
       '6TH SAN JULIAN', '18600 COLLINS ST', '8400 BEVERLY BL',
       '14100 SYLVAN ST', '11TH ST FIGUEROA ST', '12100 SANTA MONICA BL',
       '700 W 7TH ST', '18TH ST LOS ANGELES ST', '7TH ST BROADWAY',
       '200 WINSTON ST', '700 S FLOWER ST', '13600 MARINA POINT DR',
       '8TH 

In [8]:
# Dropping the 'Cross Street' column
rawDataCP.drop("cross street",axis=1,inplace=True)
rawDataCP.columns

Index(['date occ', 'time occ', 'area', 'area name', 'crm cd', 'crm cd desc',
       'vict age', 'vict sex', 'premis cd', 'premis desc', 'weapon used cd',
       'weapon desc', 'location', 'lat', 'lon'],
      dtype='object')

**3.3 Deleting rows having 'Vict Age' value <= 0**

In [9]:
rawDataCP.drop(rawDataCP[rawDataCP['vict age'] <= 0].index,inplace=True)

**3.4 Removing rows having NaN values**

In [10]:
rawDataCP.dropna(inplace=True)

In [11]:
rawDataCP

,date occ,time occ,area,area name,crm cd,crm cd desc,vict age,vict sex,premis cd,premis desc,weapon used cd,weapon desc,location,lat,lon
0,01/08/2020 12:00:00 AM,2230,3,Southwest,624,BATTERY - SIMPLE ASSAULT,36,F,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",1100 W 39TH PL,34.0141,-118.2978
1,01/01/2020 12:00:00 AM,330,1,Central,624,BATTERY - SIMPLE ASSAULT,25,M,102.0,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,700 S HILL ST,34.0459,-118.2545
5,01/01/2020 12:00:00 AM,30,1,Central,121,"RAPE, FORCIBLE",25,F,735.0,NIGHT CLUB (OPEN EVENINGS ONLY),500.0,UNKNOWN WEAPON/OTHER WEAPON,700 S BROADWAY,34.0452,-118.2534
10,01/04/2020 12:00:00 AM,2200,1,Central,330,BURGLARY FROM VEHICLE,29,M,101.0,STREET,306.0,ROCK/THROWN OBJECT,15TH OLIVE,34.0359,-118.2648
11,01/05/2020 12:00:00 AM,955,1,Central,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,35,M,108.0,PARKING LOT,511.0,VERBAL THREAT,800 N ALAMEDA ST,34.0615,-118.2412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654332,01/11/2023 12:00:00 AM,1750,1,Central,235,CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT,17,M,101.0,STREET,500.0,UNKNOWN WEAPON/OTHER WEAPON,7TH ST FLOWER ST,34.0490,-118.2592
654346,01/09/2023 12:00:00 AM,1745,4,Hollenbeck,624,BATTERY - SIMPLE ASSAULT,38,M,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",2800 CINCINNATI ST,34.0453,-118.2019
654347,01/25/2023 12:00:00 AM,100,1,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,24,F,503.0,HOTEL,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400 S FIGUEROA ST,34.0535,-118.2560
654350,01/25/2023 12:00:00 AM,400,17,Devonshire,624,BATTERY - SIMPLE ASSAULT,32,M,102.0,SIDEWALK,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",17700 LASSEN ST,34.2504,-118.5198


**3.5 Removing rows with longitude & latitude value equals 0**

In [12]:
rawDataCP.drop(rawDataCP[(rawDataCP['lon'] == 0) | (rawDataCP['lat'] == 0)].index,inplace=True)
rawDataCP

,date occ,time occ,area,area name,crm cd,crm cd desc,vict age,vict sex,premis cd,premis desc,weapon used cd,weapon desc,location,lat,lon
0,01/08/2020 12:00:00 AM,2230,3,Southwest,624,BATTERY - SIMPLE ASSAULT,36,F,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",1100 W 39TH PL,34.0141,-118.2978
1,01/01/2020 12:00:00 AM,330,1,Central,624,BATTERY - SIMPLE ASSAULT,25,M,102.0,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,700 S HILL ST,34.0459,-118.2545
5,01/01/2020 12:00:00 AM,30,1,Central,121,"RAPE, FORCIBLE",25,F,735.0,NIGHT CLUB (OPEN EVENINGS ONLY),500.0,UNKNOWN WEAPON/OTHER WEAPON,700 S BROADWAY,34.0452,-118.2534
10,01/04/2020 12:00:00 AM,2200,1,Central,330,BURGLARY FROM VEHICLE,29,M,101.0,STREET,306.0,ROCK/THROWN OBJECT,15TH OLIVE,34.0359,-118.2648
11,01/05/2020 12:00:00 AM,955,1,Central,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,35,M,108.0,PARKING LOT,511.0,VERBAL THREAT,800 N ALAMEDA ST,34.0615,-118.2412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654332,01/11/2023 12:00:00 AM,1750,1,Central,235,CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT,17,M,101.0,STREET,500.0,UNKNOWN WEAPON/OTHER WEAPON,7TH ST FLOWER ST,34.0490,-118.2592
654346,01/09/2023 12:00:00 AM,1745,4,Hollenbeck,624,BATTERY - SIMPLE ASSAULT,38,M,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",2800 CINCINNATI ST,34.0453,-118.2019
654347,01/25/2023 12:00:00 AM,100,1,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,24,F,503.0,HOTEL,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400 S FIGUEROA ST,34.0535,-118.2560
654350,01/25/2023 12:00:00 AM,400,17,Devonshire,624,BATTERY - SIMPLE ASSAULT,32,M,102.0,SIDEWALK,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",17700 LASSEN ST,34.2504,-118.5198


**3.6 Formatting time for 'time occ' column values**

In [13]:
timeOccArr = rawDataCP['time occ'].to_numpy()
newTimeOcc = []
for x in range(len(timeOccArr)):
    val = str(timeOccArr[x])
    if len(val) == 4:
        newTimeOcc.append(pd.to_datetime(str(val)[:2]+':'+str(val)[2:]).time().strftime("%H:%M:%S"))
    elif len(val) == 3:
        newTimeOcc.append(pd.to_datetime(str(val)[:1]+':'+str(val)[1:]).time().strftime("%H:%M:%S"))
    elif len(val) == 2:
        newTimeOcc.append(pd.to_datetime('0'+str(val)[:1]+':00').time().strftime("%H:%M:%S"))
    else:
        newTimeOcc.append(pd.to_datetime('0'+str(val)+':00').time().strftime("%H:%M:%S"))

rawDataCP['time occ'] = newTimeOcc
rawDataCP

,date occ,time occ,area,area name,crm cd,crm cd desc,vict age,vict sex,premis cd,premis desc,weapon used cd,weapon desc,location,lat,lon
0,01/08/2020 12:00:00 AM,22:30:00,3,Southwest,624,BATTERY - SIMPLE ASSAULT,36,F,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",1100 W 39TH PL,34.0141,-118.2978
1,01/01/2020 12:00:00 AM,03:30:00,1,Central,624,BATTERY - SIMPLE ASSAULT,25,M,102.0,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,700 S HILL ST,34.0459,-118.2545
5,01/01/2020 12:00:00 AM,03:00:00,1,Central,121,"RAPE, FORCIBLE",25,F,735.0,NIGHT CLUB (OPEN EVENINGS ONLY),500.0,UNKNOWN WEAPON/OTHER WEAPON,700 S BROADWAY,34.0452,-118.2534
10,01/04/2020 12:00:00 AM,22:00:00,1,Central,330,BURGLARY FROM VEHICLE,29,M,101.0,STREET,306.0,ROCK/THROWN OBJECT,15TH OLIVE,34.0359,-118.2648
11,01/05/2020 12:00:00 AM,09:55:00,1,Central,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,35,M,108.0,PARKING LOT,511.0,VERBAL THREAT,800 N ALAMEDA ST,34.0615,-118.2412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654332,01/11/2023 12:00:00 AM,17:50:00,1,Central,235,CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT,17,M,101.0,STREET,500.0,UNKNOWN WEAPON/OTHER WEAPON,7TH ST FLOWER ST,34.0490,-118.2592
654346,01/09/2023 12:00:00 AM,17:45:00,4,Hollenbeck,624,BATTERY - SIMPLE ASSAULT,38,M,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",2800 CINCINNATI ST,34.0453,-118.2019
654347,01/25/2023 12:00:00 AM,01:00:00,1,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,24,F,503.0,HOTEL,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400 S FIGUEROA ST,34.0535,-118.2560
654350,01/25/2023 12:00:00 AM,04:00:00,17,Devonshire,624,BATTERY - SIMPLE ASSAULT,32,M,102.0,SIDEWALK,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",17700 LASSEN ST,34.2504,-118.5198


**3.7 Converting 'date occ' column dtype to datetime**

In [14]:
rawDataCP['date occ'] = pd.to_datetime(rawDataCP['date occ'])
rawDataCP

,date occ,time occ,area,area name,crm cd,crm cd desc,vict age,vict sex,premis cd,premis desc,weapon used cd,weapon desc,location,lat,lon
0,2020-01-08,22:30:00,3,Southwest,624,BATTERY - SIMPLE ASSAULT,36,F,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",1100 W 39TH PL,34.0141,-118.2978
1,2020-01-01,03:30:00,1,Central,624,BATTERY - SIMPLE ASSAULT,25,M,102.0,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,700 S HILL ST,34.0459,-118.2545
5,2020-01-01,03:00:00,1,Central,121,"RAPE, FORCIBLE",25,F,735.0,NIGHT CLUB (OPEN EVENINGS ONLY),500.0,UNKNOWN WEAPON/OTHER WEAPON,700 S BROADWAY,34.0452,-118.2534
10,2020-01-04,22:00:00,1,Central,330,BURGLARY FROM VEHICLE,29,M,101.0,STREET,306.0,ROCK/THROWN OBJECT,15TH OLIVE,34.0359,-118.2648
11,2020-01-05,09:55:00,1,Central,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,35,M,108.0,PARKING LOT,511.0,VERBAL THREAT,800 N ALAMEDA ST,34.0615,-118.2412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654332,2023-01-11,17:50:00,1,Central,235,CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT,17,M,101.0,STREET,500.0,UNKNOWN WEAPON/OTHER WEAPON,7TH ST FLOWER ST,34.0490,-118.2592
654346,2023-01-09,17:45:00,4,Hollenbeck,624,BATTERY - SIMPLE ASSAULT,38,M,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",2800 CINCINNATI ST,34.0453,-118.2019
654347,2023-01-25,01:00:00,1,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,24,F,503.0,HOTEL,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400 S FIGUEROA ST,34.0535,-118.2560
654350,2023-01-25,04:00:00,17,Devonshire,624,BATTERY - SIMPLE ASSAULT,32,M,102.0,SIDEWALK,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",17700 LASSEN ST,34.2504,-118.5198


**3.8 Grouping data based on 'area' codes**

In [15]:
grpArea = rawDataCP.groupby(['area'])

In [16]:
grpArea.groups.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])

In [17]:
grpArea.get_group(3)

,date occ,time occ,area,area name,crm cd,crm cd desc,vict age,vict sex,premis cd,premis desc,weapon used cd,weapon desc,location,lat,lon
0,2020-01-08,22:30:00,3,Southwest,624,BATTERY - SIMPLE ASSAULT,36,F,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",1100 W 39TH PL,34.0141,-118.2978
65,2020-12-09,03:00:00,3,Southwest,236,INTIMATE PARTNER - AGGRAVATED ASSAULT,54,F,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",3000 WELLINGTON RD,34.0261,-118.3384
134,2020-09-19,10:40:00,3,Southwest,210,ROBBERY,23,M,101.0,STREET,102.0,HAND GUN,OBAMA BL VIRGINIA RD,34.0215,-118.3393
327,2020-05-29,01:25:00,3,Southwest,251,SHOTS FIRED AT INHABITED DWELLING,60,F,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",102.0,HAND GUN,2900 S REDONDO BL,34.0271,-118.3585
1053,2020-12-23,11:45:00,3,Southwest,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",63,M,122.0,"VEHICLE, PASSENGER/TRUCK",400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",BUCKINGHAM RD ADAMS BL,34.0347,-118.2809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653963,2023-01-08,23:35:00,3,Southwest,624,BATTERY - SIMPLE ASSAULT,26,F,101.0,STREET,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",1800 OBAMA BL,34.0182,-118.3133
653980,2023-01-19,18:00:00,3,Southwest,210,ROBBERY,30,M,103.0,ALLEY,119.0,MAC-10 SEMIAUTOMATIC ASSAULT WEAPON,S AUGUST ST W STEVELY AV,34.0173,-118.3536
654187,2023-01-26,16:45:00,3,Southwest,624,BATTERY - SIMPLE ASSAULT,21,F,101.0,STREET,500.0,UNKNOWN WEAPON/OTHER WEAPON,JEFFERSON BL FIGUEROA,34.0220,-118.2800
654280,2023-01-06,01:30:00,3,Southwest,624,BATTERY - SIMPLE ASSAULT,69,M,510.0,NURSING/CONVALESCENT/RETIREMENT HOME,500.0,UNKNOWN WEAPON/OTHER WEAPON,2400 S WESTERN AV,34.0340,-118.3133


### **4. Modelling**

In [18]:
# Split data into X and y
X = rawDataCP[['lat','lon']]
y = rawDataCP["area"]
#print(X)
#print(y)

In [19]:
# Split data into train and test sets
np.random.seed(42)

# Split into train & test set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
KNeighborsClassifier()
print("Predicted Values: ",knn.predict(X_test))
print("Test Values: ",y_test.to_numpy())
print('Accuracy: ',metrics.accuracy_score(y_test.to_numpy(), knn.predict(X_test)))

Predicted Values:  [17 20 11 ...  3 21 11]
Test Values:  [17 20 11 ...  3 21 11]
Accuracy:  0.9918714690948964


### **5. Using above trained model to predict the user area for further analysis. **

In [21]:
# Using Nominatim API to get the latitudes & longitudes of any user entered Address in LA. 
locator = Nominatim(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0")
location = locator.geocode(input("Enter your current address: "))
print('Location: ',location)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
areaGrpPred = knn.predict(pd.DataFrame({'lat':[location.latitude], 'lon':[location.longitude]}))[0]
print('Predicted area group: ',areaGrpPred)

Enter your current address: 1800 OBAMA BLVD
Location:  1800, Obama Boulevard, Exposition Park, Los Angeles, Los Angeles County, CAL Fire Southern Region, California, 90018, United States
Latitude = 34.018174, Longitude = -118.313404
Predicted area group:  3


In [22]:
#Finding all the co-ordinates that falls within 1km range of the above address.  
selectGrp = grpArea.get_group(areaGrpPred)
input_point = (location.latitude, location.longitude)
points = selectGrp[(selectGrp['lat'] != 0) & (selectGrp['lon'] != 0)][['date occ','location','crm cd desc','lat','lon']].to_numpy()
points_filtered = []

for point in points:
    if distance(input_point, point[3:]).km < 1:
        points_filtered.append(point)

points_filtered

[array([Timestamp('2020-05-27 00:00:00'), 'OBAMA BL COLISEUM ST',
        'BATTERY - SIMPLE ASSAULT', 34.0148, -118.3222], dtype=object),
 array([Timestamp('2020-10-11 00:00:00'), '1700 BROWNING BL',
        'CRIMINAL THREATS - NO WEAPON DISPLAYED', 34.0119, -118.3095],
       dtype=object),
 array([Timestamp('2020-01-09 00:00:00'), '1700 W 36TH ST',
        'BRANDISH WEAPON', 34.0228, -118.3089], dtype=object),
 array([Timestamp('2020-01-01 00:00:00'),
        'ARLINGTON AV MARTIN LUTHER KING JR BL',
        'INTIMATE PARTNER - SIMPLE ASSAULT', 34.0108, -118.3182],
       dtype=object),
 array([Timestamp('2020-01-01 00:00:00'), '3800 S WESTERN AV',
        'BATTERY - SIMPLE ASSAULT', 34.0146, -118.313], dtype=object),
 array([Timestamp('2020-01-01 00:00:00'), '1700 MIDDLETON PL',
        'INTIMATE PARTNER - SIMPLE ASSAULT', 34.0156, -118.311],
       dtype=object),
 array([Timestamp('2020-01-02 00:00:00'), '1500 EXPOSITION BL',
        'THEFT, PERSON', 34.0219, -118.3045], dtype=objec

In [24]:
# Further filtering data based on Age, Gender & limiting data to last six months.
six_months = datetime.now() - relativedelta( months = +6)
points_filtered = pd.DataFrame(points_filtered)
limitedData = points_filtered.loc[(points_filtered[0] >= six_months)]
#limitedData[(limitedData['vict age'] == int(age)) & (limitedData['vict sex'] == gender)]

limitedData.columns = ['date occ','location','crm cd desc','lat','lon']
limitedData

,date occ,location,crm cd desc,lat,lon
1219,2022-11-14,1700 W 36TH ST,BATTERY - SIMPLE ASSAULT,34.0228,-118.3089
1225,2022-09-12,WESTERN AV W 38TH ST,"THEFT, PERSON",34.0174,-118.3089
1226,2022-11-07,37TH ST WESTERN AV,ROBBERY,34.0211,-118.3089
1233,2022-12-27,EXPOSITION BL WESTERN AV,BRANDISH WEAPON,34.0183,-118.3089
1237,2022-09-01,1700 W MARTIN LUTHER KING JR BL,INTIMATE PARTNER - SIMPLE ASSAULT,34.0109,-118.3089
...,...,...,...,...,...
1768,2023-01-29,1600 W 39TH PL,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",34.0137,-118.3089
1769,2023-01-13,1500 W MARTIN LUTHER KING JR BL,BATTERY - SIMPLE ASSAULT,34.0108,-118.3135
1770,2023-01-24,JEFFERSON BL ST ANDREWS PL,ROBBERY,34.0256,-118.3120
1771,2023-01-04,1600 W 38TH ST,VANDALISM - MISDEAMEANOR ($399 OR UNDER),34.0174,-118.3089


In [43]:
# plotting graph
mapbox_access_token = 'pk.eyJ1IjoicnV0dmljazA3IiwiYSI6ImNsZTRhanFxNDAxNWYzcm1zN2RlcGp4YWoifQ.3EO7G6mrAtvRfRmCG_C3Kw'
gp_lat = pd.DataFrame(limitedData)['lat']
gp_lon = pd.DataFrame(limitedData)['lon']
fig = go.Figure(go.Scattermapbox(
        name='Crime Locations',
        lat=gp_lat,
        lon=gp_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=9,
            color='rgb(255, 0, 0)',
            opacity=0.6
        ),
        text= '<b>Location :</b> ' + limitedData['location'] + '<br><b>Crime Type :</b> ' + limitedData['crm cd desc'] + '<br><b>Date :</b> ' + limitedData['date occ'].dt.strftime('%d/%m/%Y')
    ))

fig.add_trace(go.Scattermapbox(
        name='User Location',
        lat=[location.latitude],
        lon=[location.longitude],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=20,
            color='rgb(0,0,255)',
            opacity=0.7
        ),
        text=location.address
    ))

fig.update_layout(
    title='Location points of cases in last 6 months within 1km of user current location.',
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=location.latitude,
            lon=location.longitude
        ),
        pitch=0,
        zoom=13.5
    ),
)

fig.show()